In [ ]:
from __future__ import print_function # python2 compatibility
import asyncio
import unittest

from hojong_py_util.util import parse_url # git submodule
from ircclient import IRCClient
from cb_mongodb import * # you can test callback functions, run the cb_*.py file

#################### twitch IRC server info ####################
SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

#################### freenode irc server info example ####################
#'''
from random import random

SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)
PASS = None
#'''

#################### callback ####################
def print_chat(nick, chat): # callback
    print('*** [RECEIVED] %s: %s' % (nick, chat))

#################### main ####################
def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** [SETTING_INFO] %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
    # if you want to use other callback, change the callback parameter
    client = IRCClient(CHANNEL, NICK, PASS, init_mongodb, log_to_mongodb, close_mongodb, loop)
    coroutine = loop.create_connection(lambda: client, host, port)

    try:
        loop.run_until_complete(coroutine) # connect
        # bot send chatting
        client.send_chat('hello, I am a bot!')
        loop.run_forever() # data receiving
    except KeyboardInterrupt as e:
        print(e)
    finally:
        loop.close()

if __name__ == '__main__':
    main()